In [5]:
pip install librosa ffmpeg-python


Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import numpy as np
import json
from collections import Counter

# Function to extract features and label from the files
def extract_features_and_label(base_path):
    try:
        # Load pitch and tonic files
        pitch_file = base_path + ".pitch.txt"
        tonic_file = base_path + ".ctonic.txt"
        json_file = base_path + ".json"

        # Check if all necessary files are available
        if not os.path.exists(pitch_file) or not os.path.exists(tonic_file) or not os.path.exists(json_file):
            return None, None

        # Load pitch data
        pitch_data = np.loadtxt(pitch_file)
        freqs = pitch_data[:, 1]
        freqs = np.where(freqs == 0, np.nan, freqs)

        # Load tonic data
        with open(tonic_file) as f:
            tonic = float(f.read().strip())

        # Normalize pitch to cents relative to tonic
        norm_pitch = 1200 * np.log2(freqs / tonic)
        norm_pitch = norm_pitch[~np.isnan(norm_pitch)]

        if len(norm_pitch) < 100:  # Skip if too short or empty
            return None, None

        # Histogram feature in the range of -600 to 1200 cents
        hist, _ = np.histogram(norm_pitch, bins=60, range=(-600, 1200), density=True)

        # Load raga label from JSON file
        with open(json_file) as f:
            meta = json.load(f)
        raga = meta["raaga"][0]["name"]

        return hist, raga

    except Exception as e:
        print(f"Error processing {base_path}: {e}")
        return None, None

# Main function to process all files in subdirectories
def load_data_from_directory(data_dir):
    X, y = [], []

    # Traverse all subdirectories
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".json"):  # Process only .json files
                base_name = os.path.splitext(file)[0]  # Remove .json extension
                base_path = os.path.join(root, base_name)  # Full path to the base

                print(f"Processing {base_path}")  # For debugging

                features, label = extract_features_and_label(base_path)
                if features is not None:
                    X.append(features)
                    y.append(label)

    return np.array(X), np.array(y)

# Directory where your dataset is stored
DATA_DIR = "D:/carnatic/carnatic"  # Make sure to set the correct path

# Load the data
X, y = load_data_from_directory(DATA_DIR)

# Check if any data was loaded
if len(X) == 0 or len(y) == 0:
    print("❌ No valid samples found. Please check the dataset directory and ensure all required files are present.")
else:
    print(f"✅ Total samples: {len(X)}")
    print(f"🎵 Unique ragas: {len(set(y))}")
    print(f"🎯 Raga counts: {Counter(y)}")


Processing D:/carnatic/carnatic\0\Cherthala Ranganatha Sharma - Varashiki Vahana
Error processing D:/carnatic/carnatic\0\Cherthala Ranganatha Sharma - Varashiki Vahana: list index out of range
Processing D:/carnatic/carnatic\1\Cherthala Ranganatha Sharma - Bhuvini Dasudane
Processing D:/carnatic/carnatic\10\Sanjay Subrahmanyan - Pullum Silambena Kan
Processing D:/carnatic/carnatic\100\Modhumudi Sudhakar - Telisi Rama
Processing D:/carnatic/carnatic\101\Modhumudi Sudhakar - Sundari Nee Divya
Processing D:/carnatic/carnatic\102\Modhumudi Sudhakar - Rama Namam Bhajare
Processing D:/carnatic/carnatic\103\Modhumudi Sudhakar - Ghandhamu Poyyaruga
Processing D:/carnatic/carnatic\104\Modhumudi Sudhakar - Mangalam Avanisutanatha
Processing D:/carnatic/carnatic\105\Mahati - Sharanagatha Vatsale
Processing D:/carnatic/carnatic\106\Mahati - Hiranmayeem
Processing D:/carnatic/carnatic\107\Mahati - Kannallavo Swami
Processing D:/carnatic/carnatic\108\Mahati - Padavini
Processing D:/carnatic/carnatic

In [ ]:
import os
import numpy as np
import json
import librosa
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")

# ========== Feature Extraction ==========
def extract_features_and_label(base_path):
    try:
        pitch_file = base_path + ".pitch.txt"
        tonic_file = base_path + ".ctonic.txt"
        json_file = base_path + ".json"

        if not os.path.exists(pitch_file) or not os.path.exists(tonic_file) or not os.path.exists(json_file):
            return None, None

        pitch_data = np.loadtxt(pitch_file)
        freqs = pitch_data[:, 1]
        freqs = np.where(freqs == 0, np.nan, freqs)

        with open(tonic_file) as f:
            tonic = float(f.read().strip())

        norm_pitch = 1200 * np.log2(freqs / tonic)
        norm_pitch = norm_pitch[~np.isnan(norm_pitch)]

        if len(norm_pitch) < 100:
            return None, None

        hist, _ = np.histogram(norm_pitch, bins=60, range=(-600, 1200), density=True)

        with open(json_file) as f:
            meta = json.load(f)

        raga = meta.get("raaga", [{}])[0].get("name")
        if not raga:
            return None, None

        return hist, raga

    except:
        return None, None

# ========== Dataset Loader ==========
def load_data_from_directory(data_dir):
    X, y = [], []
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".json"):
                base_name = os.path.splitext(file)[0]
                base_path = os.path.join(root, base_name)
                features, label = extract_features_and_label(base_path)
                if features is not None:
                    X.append(features)
                    y.append(label)
    return np.array(X), np.array(y)

# ========== Predict from Audio ==========
def predict_raga_from_audio(audio_path, clf):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
        pitch_values = []

        for i in range(pitches.shape[1]):
            index = magnitudes[:, i].argmax()
            pitch = pitches[index, i]
            if pitch > 0:
                pitch_values.append(pitch)

        pitch_values = np.array(pitch_values)
        if len(pitch_values) < 100:
            print("⚠️ Audio too short or unclear for prediction.")
            return

        tonic_hist, tonic_bins = np.histogram(np.log2(pitch_values), bins=100)
        tonic_guess = 2 ** tonic_bins[np.argmax(tonic_hist)]

        norm_pitch = 1200 * np.log2(pitch_values / tonic_guess)
        hist, _ = np.histogram(norm_pitch, bins=60, range=(-600, 1200), density=True)

        raga = clf.predict([hist])[0]
        print(f"\n🎧 Predicted Raga: {raga}")

    except Exception as e:
        print(f"❌ Error: {e}")

# ========== Main ==========
DATA_DIR = "D:/carnatic/carnatic"  # Change this to your Saraga Carnatic folder
X, y = load_data_from_directory(DATA_DIR)

if len(X) > 0:
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X, y)

    # Ask user for audio file path
    audio_path = input("🎵 Enter path to your audio file (.mp3 or .wav): ").strip()
    if os.path.exists(audio_path):
        predict_raga_from_audio(audio_path, clf)
    else:
        print("❌ File not found. Please check the path.")
else:
    print("❌ Failed to train the model. Please check your dataset path and format.")
